In [5]:
#Se importan librerias y objetos de scrapy
import scrapy
import numpy as np
from scrapy.item import Item, Field
class BookingItem(Item):
    nombre = Field()
    nota = Field()
    lugar = Field()

In [20]:
#Pipeline para guardar a json y verificar datos extraidos
import json
class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('bookingresult.jl', 'w')
    def close_spider(self, spider):
        self.file.close()
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [29]:
#Con crochet se puede construir un Crawler (Spider) que no dependa del Kernel, y se puede ejecutar las veces que quiera
from scrapy.crawler import CrawlerRunner
try:
    import crochet
except:
    !pip install crochet
    import crochet
from crochet import setup, wait_for

#Fechas de entrada y salida para determinar precio de hotel
checkin = '2022-06-01'
checkout = '2022-06-07'
#Numero de habitaciones, adultos y niños
nrooms = str(1)
nadults= str(2)
nchildren = str(0)

setup()
class BookingSpider(scrapy.Spider):
    name = "booking"
    allowed_domains = ["booking.com"]
    #Booking.com puede desplegar un maximo de 40 paginas de 25 hoteles = maximo 1000 hoteles
    #Si buscamos todos los hoteles en Chile sin filtros, encuentra aprox. 6700 alojamientos, pero solo se indexan 1000
    #En este caso se buscan todos los hoteles en Chile ordenados por recomendación
    start_urls = ['https://www.booking.com/searchresults.es.html?label=gen173nr-1FCAsoggI46AdIM1gEaC-IAQGYAQq4ARfIAQzYAQHoAQH4AQKIAgGoAgO4Ao_d-pMGwAIB0gIkNzFiYjA5NDMtNGFlNS00Mzg0LWI0MGYtYzU0N2Y4YjE1NTQ02AIF4AIB&sid=63bf5acf755828e0dd2b523389266447&aid=304142&ss=Chile&ssne=Chile&ssne_untouched=Chile&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=43&dest_type=country&checkin='+checkin+'&checkout='+checkout+'&group_adults='+nadults+'&no_rooms='+nrooms+'&group_children='+nchildren+'&sb_travel_purpose=leisure&offset=' + str(page) \
        for page in np.arange(0, 600, 25).tolist()] #Con numeros muy grandes la iteracion arroja TimeOut, pero no siempre
    
    #Definir pipeline y parámetros para guardar en json
    custom_settings = {
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        #Mediante la creación de un projecto scrapy es posible agregar valores al archivo json
        #ejecutando scrapy crawl project_name -o items.json -t json en la terminal.
        #En este caso debe utilizarse overwrite puesto que si es False, la lectura del json tira error.
        'FEEDS': {
            'booking.json': {
                'format': 'json',
                'overwrite': True 
            }
        },
        #Sin este parámetro el scraping retorna NULL
        'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
    }
    def parse(self, response):
        #Iterar en todos los objetos con la clase que define un hotel
        for question in response.css('div.a1b3f50dcd.f7c6687c3d.a1f3ecff04.f996d8c258'):
              yield{ #nombre de hotel, calificacion, ubicacion, precio por estadia (con descuento) 
                  'nombre' : question.css('div.fcab3ed991.a23c043802::text').extract_first(), 
                  'nota' : question.css('div.b5cd09854e.d10a6220b4::text').extract_first(),  
                  'lugar' : question.css('span.f4bd0794db.b4273d69aa::text').extract_first(),
                  'precio': question.css('span.fcab3ed991.bd73d13072::text').extract_first(),
              }
#Funcion para ejecutar Crawler y Spider, funciona equivalentemente a scrapy crawl projectname
@wait_for(10)
def run_spider():
    """run spider with BookingSpider"""
    crawler = CrawlerRunner()
    d = crawler.crawl(BookingSpider)
    return d

In [30]:
#Ejecutar crawler sin dependencia de kernel
run_spider()

In [31]:
#Observar datos extraidos de Booking.com
#Dado que el archivo json se reemplaza cada vez que se ejecuta Spider, no es posible 
#agregar nuevos valores al json mediante Jupyter Notebook, ver nota en custom_settings.
#Si se comparan el dataframe con lo que muestra la web, no se tiene el mismo orden de hoteles.
import pandas as pd
with open('booking.json') as f:
   data = json.load(f)
pd.DataFrame(data)

,nombre,nota,lugar,precio
0,Hostal Macal,"8,9",San Clemente,$ 322.672
1,Hotel Hallef,"8,9",Puerto Natales,$ 312.264
2,Treca Rupan Lodge,"9,2",Neltume,$ 530.692
3,Endemiko,"9,4",Malalcahuello,$ 1.349.551
4,Cabañas Lovel-Van,"9,1",Curanipe,$ 431.965
...,...,...,...,...
635,Nercon Patrimonial,"8,1",Castro,$ 364.308
636,Edificio Zafiro Reñaca,"8,2",Viña del Mar,$ 572.483
637,Vip Home,"9,0","Las Condes, Santiago",$ 354.003
638,Casa Ana María,"9,5",Coquimbo,$ 390.330


In [ ]:
#Codigo para realizar extracciones diarias 
#De igual forma se puede hacer de forma manual teniendo el script abierto, ya que el Crawler no depende del kernel
try:
    import schedule
except:
    !pip install schedule
    import schedule
import time

def job():
    #Ejecuta Crawler y visualiza json
    run_spider()
    with open('booking.json') as f:
       data = json.load(f)
    pd.DataFrame(data)
schedule.every().day.at("14:00").do(job) #se ejecuta todos los dias a las 14:00
try:
    while True:
        schedule.run_pending()
        time.sleep(1)
except KeyboardInterrupt:
    #Detener al interrumpir Kernel
    pass